In [9]:
!pip install psycopg2

In [10]:
import os
import csv

def generate_sql(config):
    sql_content = ""
    for table_config in config:
        table_name = table_config["table"]
        fields = ", ".join([f.split()[0] for f in table_config["fields"]])
        create_table_statement = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(table_config['fields'])});\n"

        # Add the CREATE TABLE statement to the SQL file content
        sql_content += create_table_statement

        # Read data from CSV and generate INSERT statements
        with open(table_config["csv_file"], newline='') as csvfile:
            reader = csv.reader(csvfile)
            next(reader)  # Skip the header row
            for row in reader:
                values = []
                for value in row:
                    # Check if the value is numeric
                    if value.isdigit():
                        values.append(value)
                    else:
                        # Properly escape single quotes in the value and enclose it in single quotes
                        escaped_value = value.replace("'", "''")
                        values.append(f"'{escaped_value}'")
                insert_statement = f"INSERT INTO {table_name} ({fields}) VALUES ({', '.join(values)});\n"
                sql_content += insert_statement

    # Ensure the output directory exists for a single output file
    output_file = "../initdb/combined_sql_output.sql"
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Write the combined SQL content to a single output file
    with open(output_file, "w") as file:
        file.write(sql_content)
    print(f"Combined SQL written to {output_file}")


In [11]:
config = [
    {
        "table": "logistics_operators",
        "csv_file": "logistics_operators.csv",
        "fields":[
            "id INT PRIMARY KEY",
            "Operator VARCHAR(255)",
            "Team CHAR(1)"
        ]
    },
    {
        "table": "logistics_vehicles",
        "csv_file": "logistics_vehicles.csv",
        "fields":[
            "id INT PRIMARY KEY",
            "Vehicle VARCHAR(255)",
            "Brand VARCHAR(255)",
            "Purchased INT"
        ]
    }
]

generate_sql(config)


Combined SQL written to ../initdb/combined_sql_output.sql
